In [1]:
import pyomo.environ as pe  # Pyomo environment
from idaes.core import FlowsheetBlock
from idaes.models.unit_models import HeatExchanger
from idaes.models.unit_models.heat_exchanger import HX0DInitializer
from idaes.models.unit_models.heat_exchanger import delta_temperature_amtd_callback
from idaes.models.properties import iapws95
import numpy as np


In [2]:
# Create an empty flowsheet and steam property parameter block.
model = pe.ConcreteModel()
model.fs = FlowsheetBlock(dynamic=False)
model.fs.params = iapws95.Iapws95ParameterBlock()

temperatures taken from AMON SharePoint AlfaLaval

In [3]:
# Provided data in Celsius, convert to Kelvin
#corrected values
T_hot_inlet_C = 691
T_hot_outlet_C = 98.248537
T_cold_inlet_C = 43.875864 
T_cold_outlet_C = 628.74162


T_hot_inlet = T_hot_inlet_C + 273.15  # Convert to Kelvin
T_hot_outlet = T_hot_outlet_C + 273.15  # Convert to Kelvin
T_cold_inlet = T_cold_inlet_C + 273.15  # Convert to Kelvin
T_cold_outlet = T_cold_outlet_C + 273.15  # Convert to Kelvin


In [4]:

# Assume constant Cp for gases (ideal gas assumption)
Cp_hot = 1.039  # Specific heat of nitrogen (N2) in kJ/kg·K
Cp_cold = 1.005  # Specific heat of air at constant pressure in kJ/kg·K


In [5]:

# Reference temperature (0°C = 273.15 K)
T_ref = 273.15

In [6]:
# Calculate enthalpy using Cp (simplified)
def calc_enthalpy(Cp, T):
    return Cp * (T - T_ref)

In [7]:
# Calculate enthalpy for the hot and cold streams
h_hot_inlet = calc_enthalpy(Cp_hot, T_hot_inlet)  # Inlet enthalpy for the hot stream (N2/steam)
h_hot_outlet = calc_enthalpy(Cp_hot, T_hot_outlet)
h_cold_inlet = calc_enthalpy(Cp_cold, T_cold_inlet)  # Inlet enthalpy for the cold stream (air)
h_cold_outlet = calc_enthalpy(Cp_cold, T_cold_outlet)



In [8]:
%store h_cold_inlet
%store h_cold_outlet
%store h_hot_inlet
%store h_hot_outlet


Stored 'h_cold_inlet' (float)
Stored 'h_cold_outlet' (float)
Stored 'h_hot_inlet' (float)
Stored 'h_hot_outlet' (float)


Look into molar flow rates

In [9]:
# Flow rates taken from AMON SharePoint AlfaLaval excel 8kW 100%
flow_mol_hot = 0.45644096  # N2 mol/s
flow_mol_cold = 0.46741802  # Air mol/s

In [10]:
# Convert molar flow to mass flow (assuming ideal gas behavior)
M_hot = 28.0134  # Molar mass of N2 (kg/kmol)
M_cold = 28.97  # Molar mass of air (kg/kmol)

In [11]:
mass_flow_hot = flow_mol_hot * M_hot / 1000  # Convert to kg/s
mass_flow_cold = flow_mol_cold * M_cold / 1000  # Convert to kg/s

In [12]:
# Print the mass flow rates for hot and cold streams
print(f"Mass flow rate of hot stream: {mass_flow_hot:.4f} kg/s")
print(f"Mass flow rate of cold stream: {mass_flow_cold:.4f} kg/s")

Mass flow rate of hot stream: 0.0128 kg/s
Mass flow rate of cold stream: 0.0135 kg/s


In [13]:
# Compute heat duty (Q)
Q_hot = mass_flow_hot * Cp_hot * (T_hot_inlet - T_hot_outlet)  # kJ/s = kW
Q_cold = mass_flow_cold * Cp_cold * (T_cold_outlet - T_cold_inlet)  # kJ/s = kW

In [14]:
# Calculate Q_avg as the average of Q_hot and Q_cold
Q_avg = (Q_hot + Q_cold) / 2

# Print Q_hot, Q_cold, and Q_avg values
print(f"Q_hot: {Q_hot:.2f} kW")
print(f"Q_cold: {Q_cold:.2f} kW")
print(f"(Q_hot + Q_cold)/2: {Q_avg:.2f} kW")

Q_hot: 7.87 kW
Q_cold: 7.96 kW
(Q_hot + Q_cold)/2: 7.92 kW


In [15]:
# Ensure heat balance is satisfied
Q = min(Q_hot, Q_cold) * 1000  # Convert kW to W

In [16]:
# Print the computed Q value
print(f"Computed Q: {Q/1000:.2f} kW")

Computed Q: 7.87 kW


In [17]:
# Compute LMTD
delta_T1 = T_hot_inlet - T_cold_outlet
delta_T2 = T_hot_outlet - T_cold_inlet
LMTD = (delta_T1 - delta_T2) / np.log(delta_T1 / delta_T2)
# Print the computed LMTD
print(f"delta_T1: {delta_T1:.2f} K")
print(f"delta_T2: {delta_T2:.2f} K")
print(f"Computed LMTD: {LMTD:.2f} K")

delta_T1: 62.26 K
delta_T2: 54.37 K
Computed LMTD: 58.23 K


In [18]:
# Fixed overall heat transfer coefficient from Cinti et al 2020
U = 30  # W/m²·K

In [19]:

# Compute heat exchanger area
A = Q / (U * LMTD)

In [20]:
print(f"Computed Heat Exchanger Area: {A:.2f} m²")

Computed Heat Exchanger Area: 4.51 m²


In [21]:
%store A

Stored 'A' (float64)


UP UNTIL HERE JUST CALCULATION, FROM HERE DOWN INFO TO VISUALIZE THE FLOWSHEET WITH UI

In [22]:
# **Store these values as variables for visualization**
model.fs.heat_duty = pe.Param(initialize=Q, doc="Heat Duty (W)")  # Heat Duty
model.fs.LMTD = pe.Param(initialize=LMTD, doc="Log Mean Temperature Difference (K)")  # LMTD
model.fs.area_calculated = pe.Param(initialize=A, doc="Calculated Heat Exchanger Area (m²)")

In [23]:
# Create the Heat Exchanger model
model.fs.heat_exchanger = HeatExchanger(
    delta_temperature_callback=delta_temperature_amtd_callback,
    hot_side_name="shell",
    cold_side_name="tube",
    shell={"property_package": model.fs.params},
    tube={"property_package": model.fs.params}
)

In [24]:
model.fs.heat_exchanger.area = A  # Set the calculated area to the heat exchanger model

In [25]:
# Fixing inlet flow and pressure values (assuming standard conditions)
model.fs.heat_exchanger.shell_inlet.flow_mol.fix(0.45644096)  # Hot stream flow rate (mol/s)
model.fs.heat_exchanger.shell_inlet.pressure.fix(104294.91)  
model.fs.heat_exchanger.tube_inlet.flow_mol.fix(0.46741802)  # Cold stream flow rate (mol/s)
model.fs.heat_exchanger.tube_inlet.pressure.fix(110546.61)  

In [26]:
# Fixing enthalpies for the heat exchanger
model.fs.heat_exchanger.shell_inlet.enth_mol[0].set_value(h_hot_inlet)  # Hot stream inlet enthalpy
model.fs.heat_exchanger.shell_outlet.enth_mol[0].set_value(h_hot_outlet)  # Hot stream outlet enthalpy
model.fs.heat_exchanger.tube_inlet.enth_mol[0].set_value(h_cold_inlet)  # Cold stream inlet enthalpy
model.fs.heat_exchanger.tube_outlet.enth_mol[0].set_value(h_cold_outlet)  # Cold stream outlet enthalpy

In [27]:

# Fix inlet enthalpies for the heat exchanger
model.fs.heat_exchanger.shell_inlet.enth_mol.fix(h_hot_inlet)  # Hot stream inlet enthalpy
model.fs.heat_exchanger.tube_inlet.enth_mol.fix(h_cold_inlet)  # Cold stream inlet enthalpy


In [28]:

# Ensure all necessary variables are fixed before initialization
model.fs.heat_exchanger.shell_inlet.flow_mol.fix(flow_mol_hot)  # Fix hot stream flow rate
model.fs.heat_exchanger.tube_inlet.flow_mol.fix(flow_mol_cold)  # Fix cold stream flow rate
model.fs.heat_exchanger.shell_inlet.enth_mol.fix(h_hot_inlet)  # Fix hot stream inlet enthalpy
model.fs.heat_exchanger.tube_inlet.enth_mol.fix(h_cold_inlet)  # Fix cold stream inlet enthalpy


In [29]:

# Fix outlet enthalpies for the heat exchanger
model.fs.heat_exchanger.shell_outlet.enth_mol[0].fix(h_hot_outlet)  # Hot stream outlet enthalpy
model.fs.heat_exchanger.tube_outlet.enth_mol[0].fix(h_cold_outlet)  # Cold stream outlet enthalpy

# Ensure all necessary variables are fixed before initialization
model.fs.heat_exchanger.shell_inlet.flow_mol[0].fix(flow_mol_hot)  # Fix hot stream flow rate
model.fs.heat_exchanger.tube_inlet.flow_mol[0].fix(flow_mol_cold)  # Fix cold stream flow rate
model.fs.heat_exchanger.shell_inlet.enth_mol[0].fix(h_hot_inlet)  # Fix hot stream inlet enthalpy
model.fs.heat_exchanger.tube_inlet.enth_mol[0].fix(h_cold_inlet)  # Fix cold stream inlet enthalpy

# Ensure outlet enthalpies are fixed before initialization
model.fs.heat_exchanger.shell_outlet.enth_mol[0].fix(h_hot_outlet)  # Fix hot stream outlet enthalpy
model.fs.heat_exchanger.tube_outlet.enth_mol[0].fix(h_cold_outlet)  # Fix cold stream outlet enthalpy


In [34]:

# Ensure all necessary variables are fixed before initialization
model.fs.heat_exchanger.shell_inlet.flow_mol[0].fix(flow_mol_hot)  # Fix hot stream flow rate
model.fs.heat_exchanger.tube_inlet.flow_mol[0].fix(flow_mol_cold)  # Fix cold stream flow rate
model.fs.heat_exchanger.shell_inlet.enth_mol[0].fix(h_hot_inlet)  # Fix hot stream inlet enthalpy
model.fs.heat_exchanger.tube_inlet.enth_mol[0].fix(h_cold_inlet)  # Fix cold stream inlet enthalpy

# Fix outlet enthalpies for initialization
model.fs.heat_exchanger.shell_outlet.enth_mol[0].fix(h_hot_outlet)  # Fix hot stream outlet enthalpy
model.fs.heat_exchanger.tube_outlet.enth_mol[0].fix(h_cold_outlet)  # Fix cold stream outlet enthalpy

# Initialize the model
initializer = HX0DInitializer()
initializer.initialize(model.fs.heat_exchanger)

KeyError: None

In [31]:
import idaes_ui
import idaes_ui.fv
dir(idaes_ui.fv)
from idaes_ui.fv import visualize 



In [32]:

# Visualize the model
visualize(model.fs, "HeatExchangerAreaCalculation")

2025-04-16 16:03:55 [INFO] idaes.idaes_ui.fv.fsvis: Started visualization server
2025-04-16 16:03:55 [INFO] idaes.idaes_ui.fv.fsvis: Loading saved flowsheet from 'HeatExchangerAreaCalculation.json'
2025-04-16 16:03:55 [INFO] idaes.idaes_ui.fv.fsvis: Saving flowsheet to default file 'HeatExchangerAreaCalculation.json' in current directory (c:\Users\Sara\Desktop\IDAES)
2025-04-16 16:03:55 [INFO] idaes.idaes_ui.fv.fsvis: Flowsheet visualization at: http://localhost:53759/app?id=HeatExchangerAreaCalculation


VisualizeResult(store=<idaes_ui.fv.persist.FileDataStore object at 0x0000020629667130>, port=53759, server=<idaes_ui.fv.model_server.FlowsheetServer object at 0x00000206296664A0>)

In [35]:
# Print results
print(f"Heat Duty (Q): {Q / 1000:.2f} kW")
print(f"LMTD: {LMTD:.2f} K")
print(f"Calculated Heat Exchanger Area: {A:.2f} m²")

Heat Duty (Q): 4.63 kW
LMTD: 58.23 K
Calculated Heat Exchanger Area: 2.65 m²
